### Задание 1
Обучите нейронную сеть решать шифр Цезаря.

Что необходимо сделать:

1. Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)
2. Сделать нейронную сеть
3. Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)
4. Проверить качество

In [40]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
import time
import re
import random
import copy

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [6]:
def caesar(string, num):
    output = ''
    for c in string:
        if c.isalpha():
            new_num = ord(c) + num
            if new_num > ord('z'):
                new_num -= 26
            output += chr(new_num)
        else:
            output += c
    return output

In [23]:
df = pd.read_csv('/content/simpsons_script_lines.csv').iloc[:,[-2]]
df = df.dropna(subset=['normalized_text'])

df['shift']   = [random.randint(1, 3) for i in range(len(df))]

# исходная реплика
df['text_in'] = [' '.join(re.findall('[\w]+', i)) for i in df['normalized_text']]

# зашифрованная реплика
df['text_out']= df.loc[:, ['text_in', 'shift']].apply(
                        lambda row: caesar(row['text_in'], row['shift']), axis=1)

df = df.iloc[:,1:]
df.head()

<ipython-input-23-fa5fe16e5b4b>:1: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/simpsons_script_lines.csv').iloc[:,[-2]]


,shift,text_in,text_out
0,2,no actually it was a little of both sometimes ...,pq cevwcnna kv ycu c nkvvng qh dqvj uqogvkogu ...
1,3,wheres mr bergstrom,zkhuhv pu ehujvwurp
2,3,i dont know although id sure like to talk to h...,l grqw nqrz dowkrxjk lg vxuh olnh wr wdon wr k...
3,3,that life is worth living,wkdw olih lv zruwk olylqj
4,3,the polls will be open from now until the end ...,wkh sroov zloo eh rshq iurp qrz xqwlo wkh hqg ...


In [26]:
MAX_LEN = 50
col_name = ''.join(['text_in_', str(MAX_LEN), '_symb'])
df[col_name] = [df.loc[i,'text_in'][:MAX_LEN] for i in df.index]

In [28]:
df['text_in_len'] =[len(df.loc[i,'text_in']) for i in df.index]
df = df[df.text_in_len > 10]
df.head()

,shift,text_in,text_out,text_in_50_symb,text_in_len
0,2,no actually it was a little of both sometimes ...,pq cevwcnna kv ycu c nkvvng qh dqvj uqogvkogu ...,no actually it was a little of both sometimes ...,151
1,3,wheres mr bergstrom,zkhuhv pu ehujvwurp,wheres mr bergstrom,19
2,3,i dont know although id sure like to talk to h...,l grqw nqrz dowkrxjk lg vxuh olnh wr wdon wr k...,i dont know although id sure like to talk to h...,100
3,3,that life is worth living,wkdw olih lv zruwk olylqj,that life is worth living,25
4,3,the polls will be open from now until the end ...,wkh sroov zloo eh rshq iurp qrz xqwlo wkh hqg ...,the polls will be open from now until the end ...,163


In [29]:
train, test = train_test_split(df, test_size=0.2)
train_text = [[c for c in ph] for ph in train['text_out'] if type(ph) is str]
train_label= [[c for c in ph] for ph in train['text_in'] if type(ph) is str]
test_text   = [[c for c in ph] for ph in test['text_out'] if type(ph) is str]
test_label  = [[c for c in ph] for ph in test['text_in'] if type(ph) is str]

In [30]:
ABC = ['none'] + [w for w in set('abcdefghijklmnopqrstuvwxyz ')]
CHAR_TO_INDEX = {w: i for i, w in enumerate(ABC)}
INDEX_TO_CHAR = {i: w for i, w in enumerate(ABC)}

def convert_to_torch(text):
    output = torch.zeros((len(text), MAX_LEN), dtype=int)
    for i in range(len(text)):
        for j, w in enumerate(text[i]):
            if j >= MAX_LEN:
                break
            output[i, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

    return output

X_train= convert_to_torch(train_text)
Y_train= convert_to_torch(train_label)
X_test = convert_to_torch(test_text)
Y_test = convert_to_torch(test_label)

In [31]:
class RNN_Network(torch.nn.Module):
    def __init__(self):
        super(RNN_Network, self).__init__()
        self.embeddings = torch.nn.Embedding(len(ABC), 28)
        self.rnn = torch.nn.RNN(28, 256, batch_first=True)
        self.linear = torch.nn.Linear(256, 28)

    def forward(self, sentences, state=None):
        embds = self.embeddings(sentences)
        out, new_state = self.rnn(embds, state)
        result = self.linear(out)
        return result, new_state

In [33]:
model = RNN_Network().to(device)
criterion = torch.nn.CrossEntropyLoss()  # типичный лосс многоклассовой классификации
optimizer = torch.optim.SGD(model.parameters(), lr=.05)

In [41]:
loss_best = 10**10
for epoch in range(20):
    start = time.time()
    train_loss = 0.
    train_passed = 0
    test_loss = 0
    test_passed = 0

    for i in range(int(len(X_train) / 100)):
        X_batch = X_train[i * 100:(i + 1) * 100].to(device)
        Y_batch = Y_train[i * 100:(i + 1) * 100].flatten().to(device)
        model.train()
        optimizer.zero_grad()
        answers, _ = model.forward(X_batch)
        answers = answers.view(-1, len(ABC))
        loss = criterion(answers, Y_batch).to(device)

        if loss < loss_best:
            model_best = copy.copy(model)#.to(dev)
            loss_best = loss#.to(dev)

        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_passed += 1

    with torch.no_grad():
        model.eval()
        answers, _ = model.forward(X_test.to(device))
        answers = answers.view(-1, len(ABC))
        loss = criterion(answers, Y_test.flatten().to(device))
        test_loss += loss.item()
        test_passed += 1

    if epoch%1 == 0:
        print(f"Epoch {epoch}. Time: {time.time() - start:.3f}, Train loss: {train_loss / train_passed:.3f}, Test loss: {test_loss / test_passed:.6f}")

Epoch 0. Time: 2.897, Train loss: 0.044, Test loss: 0.041149
Epoch 1. Time: 2.735, Train loss: 0.066, Test loss: 0.081037
Epoch 2. Time: 2.754, Train loss: 0.087, Test loss: 0.058481
Epoch 3. Time: 3.027, Train loss: 0.051, Test loss: 0.045468
Epoch 4. Time: 2.833, Train loss: 0.043, Test loss: 0.041008
Epoch 5. Time: 2.775, Train loss: 0.040, Test loss: 0.038177
Epoch 6. Time: 2.787, Train loss: 0.039, Test loss: 0.036997
Epoch 7. Time: 2.843, Train loss: 0.035, Test loss: 0.034354
Epoch 8. Time: 3.039, Train loss: 0.036, Test loss: 0.032298
Epoch 9. Time: 2.782, Train loss: 0.032, Test loss: 0.030999
Epoch 10. Time: 2.800, Train loss: 0.030, Test loss: 0.030019
Epoch 11. Time: 2.784, Train loss: 0.030, Test loss: 0.029226
Epoch 12. Time: 3.029, Train loss: 0.031, Test loss: 0.028915
Epoch 13. Time: 2.911, Train loss: 0.028, Test loss: 0.027981
Epoch 14. Time: 2.822, Train loss: 0.028, Test loss: 0.027493
Epoch 15. Time: 2.823, Train loss: 0.027, Test loss: 0.026931
Epoch 16. Time: 2.

In [51]:
tmp = df.iloc[:,1:3].head()
tmp

,text_in,text_out
0,no actually it was a little of both sometimes ...,pq cevwcnna kv ycu c nkvvng qh dqvj uqogvkogu ...
1,wheres mr bergstrom,zkhuhv pu ehujvwurp
2,i dont know although id sure like to talk to h...,l grqw nqrz dowkrxjk lg vxuh olnh wr wdon wr k...
3,that life is worth living,wkdw olih lv zruwk olylqj
4,the polls will be open from now until the end ...,wkh sroov zloo eh rshq iurp qrz xqwlo wkh hqg ...


In [54]:
ZZZ = convert_to_torch([[c for c in ph] for ph in tmp['text_out']
                                        if type(ph) is str])
tmp['text_predict']= [''.join([INDEX_TO_CHAR[i.item()] for i
                    in model_best(ZZZ.to(device))[0][line].argmax(dim=1).detach()])
                    for line in range(tmp.shape[0])]

In [55]:
tmp

,text_in,text_out,text_predict
0,no actually it was a little of both sometimes ...,pq cevwcnna kv ycu c nkvvng qh dqvj uqogvkogu ...,oo actually it was a little of both sometimes ...
1,wheres mr bergstrom,zkhuhv pu ehujvwurp,wheres mr bergstromnonenonenonenonenonenonenon...
2,i dont know although id sure like to talk to h...,l grqw nqrz dowkrxjk lg vxuh olnh wr wdon wr k...,i dont know although id sure like to talk to h...
3,that life is worth living,wkdw olih lv zruwk olylqj,that life is worth livingnonenonenonenonenonen...
4,the polls will be open from now until the end ...,wkh sroov zloo eh rshq iurp qrz xqwlo wkh hqg ...,the polls will be open from now until the end ...


### Задание 2.
Выполнить практическую работу из лекционного ноутбука.

1. Построить RNN-ячейку на основе полносвязных слоев
2. Применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”

In [56]:
df.shape

(117984, 5)

In [57]:
X = convert_to_torch([[c for c in ph]  for ph in df['text_in'].tolist()
                                        if type(ph) is str])

In [58]:
class Network(torch.nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.embed = torch.nn.Embedding(len(CHAR_TO_INDEX), 28)
        self.rnn = torch.nn.RNN(28, 812, batch_first=True)
        self.linear = torch.nn.Linear(812, len(INDEX_TO_CHAR))

    def forward(self, sentences, state=None):
        embed = self.embed(sentences)
        o, s = self.rnn(embed)
        out = self.linear(o)
        return out


In [79]:
def generate_sentence(txt_list):

    sentence =  [c for c in txt_list]
    x = torch.zeros((1, len(sentence)), dtype=int).to(device)

    for j,w in enumerate(sentence):
        if j >= MAX_S_LEN:
            break
        x[0, j] = CHAR_TO_INDEX.get(w, CHAR_TO_INDEX['none'])

    for i in range(MAX_LEN):
        o = model(x)
        w = torch.argmax(o[-1, -1, :], keepdim=True)
        x = torch.cat([x, w.unsqueeze(0)], axis=1)
        ww = INDEX_TO_CHAR[w.item()]
        if ww == 'none':
            break

        sentence.append(ww)

    return ''.join(sentence)

In [66]:
model = Network().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=.05)
MAX_S_LEN = 100
phrase = 'i dont know although id sure '

In [81]:
for ep in range(50):
    start = time.time()
    train_loss = 0.
    train_passed = 0

    for i in range(int(len(X) / 100)):
        batch = X[i * 100:(i + 1) * 100]
        X_batch = batch[:, :-1].to(device)
        Y_batch = batch[:, 1:].flatten().to(device)

        optimizer.zero_grad()
        answers = model(X_batch)
        answers = answers.view(-1, len(INDEX_TO_CHAR))
        loss = criterion(answers, Y_batch).to(device)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
        train_passed += 1

    if ep%10 == 0:
        print("\nEpoch {}. Time: {:.3f}, Train loss: {:.3f}".format(ep, time.time() - start, train_loss / train_passed))
        s = generate_sentence(phrase)
        print(s)
    else:
        print(f"\rEpoch {ep}, loss: {train_loss / train_passed:.3f}", end='')


Epoch 0. Time: 13.565, Train loss: 10.660
i dont know although id sure fa
Epoch 9, loss: 9.969
Epoch 10. Time: 13.801, Train loss: 10.122
i dont know although id sure  w wd wd wd wd wd wd wd wd wd wd wd wd wd wd wd wd
Epoch 19, loss: 2.867
Epoch 20. Time: 13.966, Train loss: 3.217
i dont know although id sure w ppbehhhh m w m
Epoch 29, loss: 6.044
Epoch 30. Time: 13.980, Train loss: 4.432
i dont know although id sure cassss se ofount y the cassss simanky unt dass tha
Epoch 39, loss: 5.985
Epoch 40. Time: 13.950, Train loss: 5.449
i dont know although id sure ing inging to hisithisinginginathisithing ithising
Epoch 49, loss: 5.847